# 필요한 라이브러리 import

In [19]:
import os
import json
import numpy as np
import pandas as pd
from scipy import sparse as spr
import configparser
import pymysql
from itertools import chain
from collections import defaultdict,Counter
import hnswlib
import warnings
import pickle
import matplotlib.pyplot as plt
from gensim.models import Word2Vec
import seaborn as sns
from sklearn.decomposition import NMF
from itertools import repeat
from sklearn.preprocessing import MinMaxScaler
import random
from scipy.stats import skew,kurtosis
warnings.filterwarnings(action='ignore')

# 데이터 로드

In [20]:
# 노래 데이터
with open('./data/song_meta.json',encoding='utf-8-sig') as f:
    song_dict = json.load(f)
    
song_df = pd.DataFrame.from_dict(song_dict)

In [21]:
#장르 데이터
with open('./data/genre_gn_all.json',encoding='utf-8-sig') as f:
    genre_dict = json.load(f)

In [22]:
# 플레이리스트 데이터
with open('data/train.json',encoding='utf-8-sig') as f:
    train_dict = json.load(f)
    
train_df = pd.DataFrame.from_dict(train_dict)

# 태그-id간 동기화를 위한 데이터베이스 연결

## 데이터베이스 연결 정보

In [23]:
# config parser 객체 생성
config = configparser.ConfigParser()

# parser객체로 config.ini 파일 읽기
config.read('config.ini')

# 연결 config 변수저장

user = config['DEFAULT']['ADMIN_USER_NAME']
passwd = config['DEFAULT']['ADMIN_PASSWORD']
host = config['DEFAULT']['RDS_ENDPOINT']
port = config['DEFAULT']['PORT']
database = config['DEFAULT']['DEFAULT_DATABASE']

## 데이터베이스 연결 및 쿼리 실행

In [24]:
# 데이터 베이스 연결
melon_recom_db = pymysql.connect(
    host = host,
    user = user,
    password = passwd,
    database = database,
    port = int(port),
    cursorclass = pymysql.cursors.DictCursor
)

# 커서 생성 후 쿼리문 수행
melon_cursor = melon_recom_db.cursor()
melon_cursor.execute('select tag_id,tag from tag')
tag_db_list=melon_cursor.fetchall()

## tag와 tag_id 간 딕셔너리 만들기 

In [25]:
# tag를 id로
tag_to_id = {}

# id를 tag로
id_to_tag = {}

for tag_db in tag_db_list:
    tag_to_id[tag_db['tag']] = tag_db['tag_id']
    
for tag_db in tag_db_list:
    id_to_tag[tag_db['tag_id']] = tag_db['tag']

# 태그 전처리

## 태그 사용 빈도수 도출

In [26]:
# train dataframe tag 컬럼의 모든 tag들 (중복포함)
tags_all = train_df['tags'].tolist()

# 태그의 빈도수를 가진 dict, Counter 써도 됨
tags_frequency = defaultdict(int)

# 특정 tag가 나올 때마다 1더하기
for tags in tags_all:
    for tag in tags:
        tags_frequency[tag] += 1

In [27]:
# 태그 빈도의 분포를 알아보자
tag_freq = pd.DataFrame().from_dict(tags_frequency,orient="index")
tag_freq.columns=['빈도']
tag_freq.describe().T

,count,mean,std,min,25%,50%,75%,max
빈도,29160.0,16.335082,247.011075,1.0,1.0,1.0,3.0,16465.0


## 태그 사용 빈도 수의 평균 값을 기준으로 태그 전처리

In [28]:
# 빈도수로 태그를 filter

def filter_func(x):
    temp = []
    for tag in x:
        if tags_frequency[tag] >=16:
            temp.append(tag)
        else:
            pass
    return temp
            
train_df['tags'] = train_df['tags'].map(filter_func)

In [29]:
# filter된 태그의 id만 남기기
train_df['tag_ids'] = train_df['tags'].map(lambda x : [tag_to_id[v] for v in x])
train_df.head(1)

,tags,id,plylst_title,songs,like_cnt,updt_date,tag_ids
0,[락],61281,여행같은 음악,"[525514, 129701, 383374, 562083, 297861, 13954...",71,2013-12-19 18:36:19.000,[25304]


## 컬럼명 한글화 및 플레이리스트 당 태그의 수 컬럼 생성

In [30]:
train_df.columns=['태그','플리아이디','플리제목','노래들','좋아요수','변경일자','태그아이디']
train_df['태그수'] = train_df['태그'].map(len)

In [31]:
model = Word2Vec(sentences=train_df['태그'].tolist(), vector_size=100, window=100, min_count=1, workers=4)

In [32]:
model.wv.most_similar(['잔잔한','카페에서'], topn=10)

[('감성음악', 0.8657983541488647),
 ('잔잔', 0.8438810706138611),
 ('책', 0.8295848369598389),
 ('차분한', 0.8164227604866028),
 ('편안한', 0.8150824904441833),
 ('쉼', 0.8121812343597412),
 ('차분', 0.810287594795227),
 ('나른함', 0.8088546395301819),
 ('커피한잔', 0.8081028461456299),
 ('잔잔한음악', 0.8078885674476624)]

# 플레이리스트의 태그로 예측에 사용 될 태그와 검증에 사용 될 태그로 나누기

In [33]:
# 태그가 3개인거 부터 train에 쓸거임
origin_tags = train_df[train_df['태그수']>3]['태그'].tolist()

train_tags = []
test_tags = []

# 3:7비율로 나누기
for tags in origin_tags:
    tag_3p = len(tags)//3
    train_tag = random.sample(tags, tag_3p)
    test_tag = list(set(tags)-set(train_tag))
    train_tags.append(train_tag)
    test_tags.append(test_tag)
    
cal_tag_hit_df = pd.DataFrame(columns=['예측용태그','검증용태그','svd_예측결과','svd_히트'])

cal_tag_hit_df['예측용태그'] = train_tags
cal_tag_hit_df['검증용태그'] = test_tags
cal_tag_hit_df.head(1)

,예측용태그,검증용태그,svd_예측결과,svd_히트
0,"[눈오는날, 따듯한]","[겨울왕국, 크리스마스, 캐럴, 분위기, 겨울노래, 연말]",NaN,NaN


# 태그 예측

## 예측 수행 및 수행 결과 저장

In [45]:
# svd_knn_model로 태그를 예측하여 예측된 결과의 list를 저장하는 함수

def tag_w2v(x):
    
    results = model.wv.most_similar(x, topn=10+len(x))
    
    tags = [result[0] for result in results]

    return list(set(tags)-set(x))

In [46]:
# 예측수행
cal_tag_hit_df['w2v_예측결과'] = cal_tag_hit_df['예측용태그'].map(tag_w2v)

In [47]:
cal_tag_hit_df.head()

,예측용태그,검증용태그,svd_예측결과,svd_히트,w2v_예측결과
0,"[눈오는날, 따듯한]","[겨울왕국, 크리스마스, 캐럴, 분위기, 겨울노래, 연말]",NaN,NaN,"[추위, 따뜻한, 추운, 캐럴, 겨울감성, 찬바람, 첫눈, 겨울노래, 추운날, 눈,..."
1,"[팝, 트렌드, 드라이브]","[힐링, 운동, 일렉, 트로피컬하우스, Pop, 기분전환, 2017]",NaN,NaN,"[팝송추천, 국외, 힙, 도입부, 스타일리쉬, 트렌디한, 유니크, 팝송, 트렌디, ..."
2,"[이별, 짝사랑]","[취향저격, 고백, 사랑, 슬픔]",NaN,NaN,"[기억, 쓸쓸, 이별노래, 쓸쓸함, 헤어짐, 첫사랑, 그리움, 후회, 아픔, 보고싶..."
3,"[포크, 댄스]","[인디, 일렉트로니카, 락, 메탈]",NaN,NaN,"[Folk, 숨겨진명곡, 밴드, 컨트리, 밴드음악, 최애곡, 여돌, 아이돌의숨은명곡..."
4,"[락, 메탈]","[M에센셜, 록, Rock, Metal]",NaN,NaN,"[록메탈, 밴드, Metal, 브릿팝, 하드록메탈, Rock, 얼터너티브록, 얼터너..."


## 평가 지표

In [48]:
val_tag = cal_tag_hit_df['검증용태그'].tolist()
w2v_tag = cal_tag_hit_df['w2v_예측결과'].tolist()

w2v_hit = []

for val,w2v in zip(val_tag,w2v_tag):
    if len(val) == len(set(val)-set(w2v)):
        w2v_hit.append(0)
    else:
        w2v_hit.append(1)
        
cal_tag_hit_df['w2v_히트'] = w2v_hit

In [49]:
cal_tag_hit_df.head(5)

,예측용태그,검증용태그,svd_예측결과,svd_히트,w2v_예측결과,w2v_히트
0,"[눈오는날, 따듯한]","[겨울왕국, 크리스마스, 캐럴, 분위기, 겨울노래, 연말]",NaN,NaN,"[추위, 따뜻한, 추운, 캐럴, 겨울감성, 찬바람, 첫눈, 겨울노래, 추운날, 눈,...",1
1,"[팝, 트렌드, 드라이브]","[힐링, 운동, 일렉, 트로피컬하우스, Pop, 기분전환, 2017]",NaN,NaN,"[팝송추천, 국외, 힙, 도입부, 스타일리쉬, 트렌디한, 유니크, 팝송, 트렌디, ...",0
2,"[이별, 짝사랑]","[취향저격, 고백, 사랑, 슬픔]",NaN,NaN,"[기억, 쓸쓸, 이별노래, 쓸쓸함, 헤어짐, 첫사랑, 그리움, 후회, 아픔, 보고싶...",0
3,"[포크, 댄스]","[인디, 일렉트로니카, 락, 메탈]",NaN,NaN,"[Folk, 숨겨진명곡, 밴드, 컨트리, 밴드음악, 최애곡, 여돌, 아이돌의숨은명곡...",0
4,"[락, 메탈]","[M에센셜, 록, Rock, Metal]",NaN,NaN,"[록메탈, 밴드, Metal, 브릿팝, 하드록메탈, Rock, 얼터너티브록, 얼터너...",1


In [50]:
sum(w2v_hit)/len(w2v_hit)

0.33090718262298074

In [85]:
svd_inter = []

for val,svd in zip(val_tag,svd_tag):
    svd_inter.append(len(set(val) - (set(val)-set(svd))))
        
cal_tag_hit_df['svd_교집합'] = svd_inter

In [86]:
cal_tag_hit_df.head(30)

,예측용태그,검증용태그,svd_예측결과,svd_히트,svd_교집합
0,"[눈오는날, 따듯한]","[연말, 크리스마스, 캐럴, 겨울왕국, 분위기, 겨울노래]","[눈, 감성발라드, 첫눈, 크리스마스, 겨울, 잔잔한, 혼자, 추운날, Winter...",1,1
1,"[드라이브, 기분전환, 팝]","[운동, 힐링, 트로피컬하우스, Pop, 일렉, 2017, 트렌드]","[운동, 팝송모음, 스트레스, 팝송추천, 매장음악, 추천곡, 산책, popsong,...",1,2
2,"[고백, 사랑]","[취향저격, 이별, 짝사랑, 슬픔]","[데이트, 사랑노래, 달달한노래, 달달, 연애세포, 연인, 두근두근, 연애, 커플, 썸]",0,0
3,"[댄스, 락]","[포크, 메탈, 일렉트로니카, 인디]","[락음악, 밴드음악, 스트레스, Rock, Coldplay, 활력, 2000_10,...",0,0
4,"[락, 록]","[Metal, M에센셜, Rock, 메탈]","[락음악, 밴드음악, Rock, Coldplay, 2000_10, 밴드, 모던락, ...",1,1
5,"[느낌있는, 밤, 새벽]","[감각적인, RnB, 그루브한, 힙합, 드라이브, 국내]","[추천노래, 띵곡, 매장음악, 산책, 취향저격, 잔잔한, 그루브, 분위기, 비오는날...",0,0
6,[질리지않는],"[나만알고싶은, 감성, Pop]","[팝송추천, 퓨처베이스, AlanWalker, 디즈니OST, popsong, 겨울왕...",0,0
7,[새벽감성],"[혼자, 고민, 조용히]","[비, 퇴근길, 쓸쓸한, 상처, 새벽에듣기좋은노래, 쓸쓸, 잔잔한, 울고싶을때, 밤...",0,0
8,"[잔잔한, 편안한]","[밤, 감성, 인디, 어쿠스틱, 새벽]","[쓸쓸함, 쓸쓸하게, 센치, 산책, 힐링, 혼자, 집중, 그리움, 조용한, 여유]",0,0
9,[힐링],"[에너지, 여행, 인디, 드라이브]","[연주음악, 뉴에이지, 생각, 독서, 피아노, 여유로운, 책, 편안한, 연주곡, 여유]",0,0
